## Universal sentence encoder 사용 예시
* 문장에 대한 직접적인 비교가 가능
* 로컬 환경에서 실행하려면 아래 페이지를 참고하자
    * https://www.tensorflow.org/hub/basics

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

# Some texts of different lengths.
sentences = ["나는 한글을 사랑합니다",
                    "나는 우리말을 사랑합니다", 
                    "나는 너를 사랑합니다", 
                    "나는 한글이 너무 아름답다고 생각합니다", 
                    "메모장은 사용이 불편합니다", 
                    "내가 사용하던 메모장은 기능이 많이 부족합니다", 
                    "메모장은 글자를 쓸 수 있는 도구입니다", 
                    "이 메모장은 정말 훌륭한 것 같습니다",
                    "이 직원은 정말 일을 잘합니다",  
                    "은행 직원이 너무 불친절 합니다", 
                    "은행 직원이 기뻐하는 모습이 아름답습니다",
                    "예금 인출을 위해서는 먼저 줄을 서야 합니다", 
                    "버스를 타려면 저기 줄 뒤에 서세요"]

W0416 15:27:30.779164 139785420543744 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
# Set up graph.
g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    xling_8_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling-many/1")
    embedded_text = xling_8_embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0416 15:27:45.987104 139785420543744 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [7]:
# Compute embeddings.
kr_result = session.run(embedded_text, feed_dict={text_input: sentences})

# Compute similarity matrix. Higher score indicates greater similarity.
sim_mat = np.inner(kr_result, kr_result)

print(kr_result)
print(sim_mat)

[[ 0.05336764 -0.020971   -0.01621751 ... -0.09863348  0.05069829
  -0.03755969]
 [-0.01632101 -0.06642696 -0.0253969  ... -0.09832521  0.02571269
   0.02095492]
 [-0.01147083 -0.0680128  -0.0244094  ... -0.08506192 -0.01118958
   0.05937229]
 ...
 [ 0.02129305 -0.00904285  0.00223683 ... -0.09466263 -0.03796628
  -0.05667244]
 [ 0.06210278  0.00982793 -0.04973892 ... -0.03706284 -0.02946508
  -0.03796549]
 [ 0.01289104 -0.05725896 -0.00527408 ... -0.09200849 -0.03560571
  -0.00684112]]
[[1.0000002  0.65589565 0.5733584  0.7759081  0.30173162 0.24042828
  0.2609595  0.4825729  0.43518743 0.3168612  0.38949448 0.18263823
  0.16737138]
 [0.65589565 0.9999999  0.65545404 0.47148776 0.26304603 0.19167252
  0.12236132 0.40009493 0.4413885  0.3157822  0.38553917 0.139135
  0.22325863]
 [0.5733584  0.65545404 0.9999999  0.42906526 0.20599075 0.15860288
  0.12261134 0.4291544  0.426741   0.31456044 0.3739106  0.19344404
  0.3056507 ]
 [0.7759081  0.47148776 0.42906526 0.9999999  0.33596057 0.2

In [8]:
import operator
sent_dic = {sent: i for i, sent in enumerate(sentences) }
def get_sim(sim_mat, sent):
    i = sent_dic[sent]
    vals = sim_mat[i]
    val_dic = {j: val for j, val in enumerate(vals) if j is not i}
    sorted_val = sorted(val_dic.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_val

In [9]:
for sent1 in sentences:
    print(sent1)
    vals = get_sim(sim_mat, sent1)[:3]
    sim_sent = [sentences[x[0]] for x in vals]
    print('\t'+'\t'.join(sim_sent))
    print()

나는 한글을 사랑합니다
	나는 한글이 너무 아름답다고 생각합니다	나는 우리말을 사랑합니다	나는 너를 사랑합니다

나는 우리말을 사랑합니다
	나는 한글을 사랑합니다	나는 너를 사랑합니다	나는 한글이 너무 아름답다고 생각합니다

나는 너를 사랑합니다
	나는 우리말을 사랑합니다	나는 한글을 사랑합니다	이 메모장은 정말 훌륭한 것 같습니다

나는 한글이 너무 아름답다고 생각합니다
	나는 한글을 사랑합니다	이 메모장은 정말 훌륭한 것 같습니다	은행 직원이 기뻐하는 모습이 아름답습니다

메모장은 사용이 불편합니다
	내가 사용하던 메모장은 기능이 많이 부족합니다	메모장은 글자를 쓸 수 있는 도구입니다	이 메모장은 정말 훌륭한 것 같습니다

내가 사용하던 메모장은 기능이 많이 부족합니다
	메모장은 사용이 불편합니다	메모장은 글자를 쓸 수 있는 도구입니다	이 메모장은 정말 훌륭한 것 같습니다

메모장은 글자를 쓸 수 있는 도구입니다
	메모장은 사용이 불편합니다	내가 사용하던 메모장은 기능이 많이 부족합니다	이 메모장은 정말 훌륭한 것 같습니다

이 메모장은 정말 훌륭한 것 같습니다
	메모장은 사용이 불편합니다	나는 한글이 너무 아름답다고 생각합니다	이 직원은 정말 일을 잘합니다

이 직원은 정말 일을 잘합니다
	은행 직원이 기뻐하는 모습이 아름답습니다	은행 직원이 너무 불친절 합니다	이 메모장은 정말 훌륭한 것 같습니다

은행 직원이 너무 불친절 합니다
	은행 직원이 기뻐하는 모습이 아름답습니다	이 직원은 정말 일을 잘합니다	예금 인출을 위해서는 먼저 줄을 서야 합니다

은행 직원이 기뻐하는 모습이 아름답습니다
	은행 직원이 너무 불친절 합니다	이 직원은 정말 일을 잘합니다	나는 한글이 너무 아름답다고 생각합니다

예금 인출을 위해서는 먼저 줄을 서야 합니다
	은행 직원이 너무 불친절 합니다	버스를 타려면 저기 줄 뒤에 서세요	은행 직원이 기뻐하는 모습이 아름답습니다

버스를 타려면 저기 줄 뒤에 서세요
	예금 인출을 위해서는 먼저 줄을 서야 합니다	나는 너를 사랑합니다	이 